In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)   

data = pd.read_csv('DATA_DESA.csv')

df = pd.DataFrame(data)

# Membagi kolom tanggal menjadi tahun dan bulan
df['tahun'] = df['tahun_bulan'].apply(lambda x: int(x.split('-')[0]))
df['bulan'] = df['tahun_bulan'].apply(lambda x: int(x.split('-')[1]))

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Memisahkan fitur dan target
X = df[['tahun', 'bulan']].values
y = df['jumlah_kecelakaan'].values

In [3]:
# Pisahkan data menjadi training dan testing
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [4]:
# Fungsi untuk menghitung MAPE menggunakan model SVR dengan parameter tertentu
def calculate_mape(X_train, y_train, X_val, y_val, params):
    C, epsilon, gamma = params
    svr = SVR(kernel='linear', C=C, epsilon=epsilon, gamma=gamma)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_val)
    mape = mean_absolute_percentage_error(y_val, y_pred) *100
    return mape

# Inisialisasi populasi awal
def initialize_population(population_size, parameter_ranges):
    population = []
    for _ in range(population_size):
        chromosome = []
        for param_range in parameter_ranges:
            param_value = np.random.uniform(param_range[0], param_range[1])
            chromosome.append(param_value)
        population.append(chromosome)
    return population

# Seleksi individu berdasarkan nilai fitness (MAPE)
def selection(population, X_train, y_train, X_val, y_val):
    selected_population = []
    for chromosome in population:
        mape = calculate_mape(X_train, y_train, X_val, y_val, chromosome)
        selected_population.append((chromosome, mape))
    selected_population.sort(key=lambda x: x[1])  # Sort berdasarkan MAPE terendah
    return selected_population[:len(population)//2]  # Pilih setengah individu terbaik

# Crossover: Menghasilkan keturunan dari pasangan individu terpilih
def crossover(selected_population):
    new_population = []
    ukuran_populasi = len(selected_population)
    
    for i in range(ukuran_populasi):
        parent1 = selected_population[i][0]
        
        # Tentukan parent2 dengan pemeriksaan batas
        if i % 2 == 0:
            if i - 1 >= 0:
                parent2 = selected_population[i-1][0]
            else:
                parent2 = selected_population[i][0]  # Penanganan alternatif jika i-1 tidak valid
        else:
            if i + 1 < ukuran_populasi:
                parent2 = selected_population[i+1][0]
            else:
                parent2 = selected_population[i][0]  # Penanganan alternatif jika i+1 tidak valid
        
        crossover_point = np.random.randint(1, len(parent1)-1)
        child = parent1[:crossover_point] + parent2[crossover_point:]
        new_population.append(child)
    
    return new_population

# Mutasi: Mengubah nilai gen secara acak pada populasi baru
def mutation(new_population, mutation_rate, parameter_ranges, sigma_range):
    for i in range(len(new_population)):
        if np.random.random() < mutation_rate:
            mutated_gene_index = np.random.randint(0, len(new_population[i]))
            sigma = np.random.uniform(sigma_range[0], sigma_range[1])
            new_value = new_population[i][mutated_gene_index] + np.random.normal(0, sigma)
            new_population[i][mutated_gene_index] = np.clip(new_value, parameter_ranges[mutated_gene_index][0], parameter_ranges[mutated_gene_index][1])
    return new_population

In [5]:
# Rentang nilai parameter optimal
parameter_ranges = [(0.0001, 0.01), (0.001, 1), (1e-07, 0.0001)]

# Inisialisasi populasi awal
population_size = 100
population = initialize_population(population_size, parameter_ranges)

# Iterasi algoritma genetika
crossover_rate = 1  # Tetapkan tingkat crossover tetap untuk eksperimen
mutation_rate = 0  # Tetapkan tingkat mutasi tetap untuk eksperimen
num_generations = 100  # Tetapkan jumlah generasi tetap untuk eksperimen
sigma_range = (0.1, 5)

for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    selected_population = selection(population, X_train, y_train, X_val, y_val)
    for i, (chromosome, mape) in enumerate(selected_population):
        print(f"Individu {i + 1}: C={chromosome[0]}, epsilon={chromosome[1]}, gamma={chromosome[2]}, MAPE={mape}")
    
    new_population = crossover(selected_population)
    new_population = mutation(new_population, mutation_rate, parameter_ranges, sigma_range)

    # Adjust the population size if necessary
    if len(new_population) < population_size:
        additional_individuals = initialize_population(population_size - len(new_population), parameter_ranges)
        new_population.extend(additional_individuals)
    elif len(new_population) > population_size:
        new_population = new_population[:population_size]
    
    # Make sure the new population has the correct size
    assert len(new_population) == population_size, "Ukuran populasi baru tidak sesuai"
    
    population = new_population

# Mengambil individu terbaik (dengan MAPE terendah) dari populasi terakhir
best_chromosome, best_mape = min(selected_population, key=lambda x: x[1])

# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='linear', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nParameter terbaik:")
print(f"C: {best_chromosome[0]}")
print(f"Epsilon: {best_chromosome[1]}")
print(f"Gamma: {best_chromosome[2]}")
print(f"MAPE terbaik: {best_mape}")

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)

Generasi 1:
Individu 1: C=0.003947905167925873, epsilon=0.272077682742122, gamma=8.289087716427775e-05, MAPE=33.607022208858965
Individu 2: C=0.008785659598471713, epsilon=0.25868368608744047, gamma=6.603240619881449e-05, MAPE=33.61391534160775
Individu 3: C=0.003631857934266534, epsilon=0.2816535751776934, gamma=5.4315338707509025e-05, MAPE=33.69706663774566
Individu 4: C=0.004376255684556947, epsilon=0.2919379110578439, gamma=6.122410418276571e-05, MAPE=33.82217838403195
Individu 5: C=0.001915704647548995, epsilon=0.3049380007165782, gamma=5.2523167520060566e-05, MAPE=33.83716957378458
Individu 6: C=0.008341182143924175, epsilon=0.21312677156759788, gamma=1.826431422398935e-05, MAPE=33.916316015857824
Individu 7: C=0.009091832271069872, epsilon=0.24032232877630544, gamma=1.4574997721913188e-05, MAPE=33.92347140551348
Individu 8: C=0.0014809892204552142, epsilon=0.29285250388668294, gamma=3.6699548145039804e-05, MAPE=33.94215319330776
Individu 9: C=0.0031787249849850557, epsilon=0.325

In [6]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[ 7.62221876]
 [ 9.9942154 ]
 [ 8.58460467]
 [12.85419863]
 [ 8.41741339]
 [11.75179594]
 [12.53340332]
 [ 8.90539997]
 [ 7.94301406]
 [13.17499393]
 [13.32859796]
 [ 9.85419863]]


In [7]:
# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='poly', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)


Hasil peramalan:
Prediksi: [0.31209163 0.4434605  0.42507473 0.66223743 0.37601571 0.62107748
 0.6243046  0.46281033 0.3497154  0.70020767 0.68858759 0.51556705]
MAPE: 34.72062590133922


In [8]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[ 6.24183265]
 [ 8.86920998]
 [ 8.50149462]
 [13.24474857]
 [ 7.52031419]
 [12.42154959]
 [12.48609193]
 [ 9.25620653]
 [ 6.99430804]
 [14.00415341]
 [13.77175172]
 [10.31134095]]


In [9]:
# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='rbf', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)


Hasil peramalan:
Prediksi: [0.54696803 0.54704332 0.54698092 0.5471167  0.54698731 0.54707573
 0.54711237 0.54698525 0.54697231 0.54712101 0.54713174 0.54701532]
MAPE: 36.0754940190545


In [10]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[10.93936064]
 [10.94086633]
 [10.93961835]
 [10.94233394]
 [10.93974621]
 [10.9415146 ]
 [10.94224748]
 [10.93970494]
 [10.93944615]
 [10.9424202 ]
 [10.94263489]
 [10.94030632]]


In [11]:
# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='sigmoid', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)


Hasil peramalan:
Prediksi: [0.5470498 0.5470498 0.5470498 0.5470498 0.5470498 0.5470498 0.5470498
 0.5470498 0.5470498 0.5470498 0.5470498 0.5470498]
MAPE: 36.078306724187925


In [12]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[10.940996]
 [10.940996]
 [10.940996]
 [10.940996]
 [10.940996]
 [10.940996]
 [10.940996]
 [10.940996]
 [10.940996]
 [10.940996]
 [10.940996]
 [10.940996]]
